In [1]:
import three_desc_model as exponential_new
import numpy as np
import pandas as pd
from cutoffs import Polynomial
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from scipy.optimize import minimize
turbs = ["Siemens"] *2
turbs_3 = ["Siemens"] *3
nl=exponential_new.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)


symbol="Siemens"
dataset_pos=np.empty(shape=(0, 2))
dataset_fp=np.empty(shape=(0, 3))
dataset_pri=np.empty(shape=(0, 1))



In [2]:
def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [11]:
dataset = pd.read_csv('all_dataset.csv', index_col=0)
len(dataset)

1784

In [13]:
for i,row in dataset.iterrows():
    if row['Num_tot_turb'] == 3:
        position = np.array([[dataset.at[i,"X_coord"],dataset.at[i,"Y_coord"]],[dataset.at[i,"X_coord"],id_dataset.at[1,"Y_coord"]],[id_dataset.at[2,"X_coord"],id_dataset.at[2,"Y_coord"]]])

NameError: name 'id_dataset' is not defined

In [27]:
gb=dataset.groupby(["Num_tot_turb"])
#[gb.get_group(x) for x in gb.groups]
list(gb)

[(2.0,
           X_coord     Y_coord  Ref_wind  Num_tot_turb  \
  0       0.000000    0.000000  7.479762           2.0   
  1     241.621088  139.500000  7.576231           2.0   
  2       0.000000    0.000000  7.473602           2.0   
  3     244.018898  135.261884  7.567527           2.0   
  4       0.000000    0.000000  7.467848           2.0   
  ..           ...         ...       ...           ...   
  699  3499.466933  -61.083423  7.245134           2.0   
  700     0.000000    0.000000  7.483274           2.0   
  701  3495.203372 -183.175847  7.384614           2.0   
  702     0.000000    0.000000  7.433140           2.0   
  703  4000.000000    0.000000  7.183321           2.0   
  
                                       ID  Turb_num  
  0    2_turbine_240_degrees_279.0_meters       0.0  
  1    2_turbine_240_degrees_279.0_meters       1.0  
  2    2_turbine_241_degrees_279.0_meters       0.0  
  3    2_turbine_241_degrees_279.0_meters       1.0  
  4    2_turbine_242_deg

In [4]:
for i in range(len(dataset)):
    if dataset.loc[dataset['Number of Turbine in the Simulation'] == 2].reset_index(drop=True)
    position = np.array([[id_dataset.at[0,"X_coord"],id_dataset.at[0,"Y_coord"]],[id_dataset.at[1,"X_coord"],id_dataset.at[1,"Y_coord"]],[id_dataset.at[2,"X_coord"],id_dataset.at[2,"Y_coord"]]])
    ws = np.array([[id_dataset.at[0,"Ref_Wind_Speed"]], [id_dataset.at[1,"Ref_Wind_Speed"]], [id_dataset.at[2,"Ref_Wind_Speed"]]])
    neigh=nl.calculate(turbs_3,position)
    finpr=exponential_new.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
    fp=finpr.calculate(turbs_3,position,neigh,symbol)
    fp=np.array(fp)
    dataset_pos=np.append(dataset_pos,position,axis=0)
    dataset_fp=np.append(dataset_fp,fp,axis=0)
    dataset_ws=np.append(dataset_ws,ws,axis=0)
dataset=np.concatenate((dataset_fp, dataset_ws),axis=1)

NameError: name 'three_turb_dataset' is not defined

In [ ]:
def fitness(x):
    Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":x[0], "offset": 1.},
                  {"type":"G4", "elements":"Siemens","eta":x[1], "gamma":4., "zeta": 1.0},
                  {"type":"G6", "elements":"Siemens","eta":x[2], "gamma": 0.5, "zeta": 1.0}
                 ]}
    turbs_3 = ["Siemens"] *3
    nl=exponential_new.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)


    symbol="Siemens"

    
    dataset = pd.DataFrame(dataset, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Ref_Wind_Speed'])
    train_dataset = dataset.sample(frac=0.8, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)
    train_features = train_dataset.copy()
    test_features = test_dataset.copy()
    all_features = dataset.copy()
    train_labels = train_features.pop('Ref_Wind_Speed')
    test_labels = test_features.pop('Ref_Wind_Speed')
    all_labels = all_features.pop('Ref_Wind_Speed')
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(train_features))
    dnn_model = build_and_compile_model(normalizer)
    history = dnn_model.fit(
        train_features, train_labels,
        validation_split=0.2,
        verbose=0, epochs=120)
    all_predictions = dnn_model.predict(all_features).flatten()
    error_all=np.abs(all_predictions-all_labels)
    return np.max(error_all)

In [ ]:
x0 = [5.3, 5., 2.]
bounds= ((0,10),(0,10),(0,10))
res = minimize(fitness, x0, bounds= bounds, options={'disp':True})